Preparing to submit PacBio long read data for late 2018 early 2019

Link to sheet

https://docs.google.com/spreadsheets/d/1GBMmL3Mmzwe_chp-jOxEfKo7jK70B90rpA3nOz-qPyU/edit?ts=5ea1fd7b#gid=1718836302

In [1]:
import os
import sys
from pathlib import Path
import requests
import pandas
import paramiko
import json
from IPython import display

In [2]:
EC = str(Path("~/proj/encoded_client").expanduser())
if EC not in sys.path:
    sys.path.append(EC)
from encoded_client.encoded import ENCODED, DCCValidator, Document
from encoded_client.aws_submission import run_aws_cp

In [3]:
GCATDIR = os.path.expanduser('~/src/gcat')
if GCATDIR not in sys.path:
    sys.path.append(GCATDIR)
    import gcat

In [4]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = "ENCODE4 May2019 PacBio"

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [5]:
award = 'UM1HG009443'

# Submit Documents

Example Document submission

This is the wrong protocol

In [6]:
smrtbell_uuid = '/documents/2297237f-dfb0-424d-a386-c350ffe8dbe3/'
smrtbell = Document(
    os.path.expanduser('~/proj/encode4-curation/ENCODE_protocol_brain_pacbio.pdf'),
    'general protocol',
    'SMRTbell cDNA library construction protocol for PacBio sequencing',
)
body = smrtbell.create_if_needed(server, smrtbell_uuid, validator)
if '@graph' in body:
    print(body['@graph'][0]['@id'])
else:
    print(body['@id'])

/documents/2297237f-dfb0-424d-a386-c350ffe8dbe3/


# Submit Annotations

In [7]:
#sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#annotations = sheet.parse('Annotations', header=0)
#created = server.post_sheet('/annotations/', annotations, verbose=True, dry_run=True)
#print(len(created))

In [8]:
#if created:
#    annotations.to_excel('/tmp/annotations.xlsx', index=False)

# Register Biosamples

In [9]:
#book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#biosample = book.parse('Biosamples', header=0)
#
#for i, row in biosample.iterrows():
#    if pandas.isnull(row['source']) and not pandas.isnull(row['accession']):
#        obj = server.get_json(row['accession'])
#        #print(row['accession'], obj['source']['@id'])
#        biosample.loc[i, 'source'] = obj['source']['@id']
#        print(i, biosample.loc[i, 'source'])

In [10]:
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
biosample = book.parse('Biosamples', header=0)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/src/gcat/gcat/__init__.py:55: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


0


In [11]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [12]:
validator._aliases['/documents/1734f117-1ac9-4596-9977-ffd4cf0dc6b3/'] = {}

In [13]:
# In case we need to look up previously submitted libraries by alias.
if 0:
    book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
    libraries = book.parse('Libraries', header=0)    
    for i, row in libraries.iterrows():
        if pandas.isnull(row['accession']) and not pandas.isnull(row['aliases:array']):
            alias = row['aliases:array'].split(',')[0]
            obj = server.get_json(alias)
            libraries.loc[i, 'accession'] = obj['accession']

    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)
    libraries

In [23]:
print(spreadsheet_name)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = book.parse('Libraries', header=0)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

ENCODE4 May2019 PacBio
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'documents': ['/documents/3baa46d2-cb88-4608-8877-70596d200489/', '/documents/bf543ffc-23d0-4f6f-b6a8-d131859259e4/'], 'date_created': '2021-08-13T23:13:22.442880+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'status': 'in progress', 'lab': '/labs/ali-mortazavi/', 'award': '/awards/UM1HG009443/', 'aliases': ['ali-mortazavi:PB400_m_p18-20mo_heart'], 'accession': 'ENCLB607FYB', 'schema_version': '18', 'alternate_accessions': [], 'spikeins_used': ['/references/ENCSR759PLA/', '/references/ENCSR156CIL/'], 'barcode_details': [], 'biosample': '/biosamples/ENCBS540JOT/', 'nucleic_acid_term_name': 'polyadenylated mRNA', 'dbxrefs': [], 'nucleic_acid_starting_quantity': 2, 'nucleic_acid_starting_quantity_units': 'µg', 'rna_integrity_number': 8, 'strand_specificity': 'forward', 'treatments': [], '@id': '/libraries/ENCLB607FYB/', '@type': ['Library', 'Item'], 'uuid': '19f34244-1c13-4032-a

In [24]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [16]:
# In case we need to look up previously submitted libraries by alias.
if 0:
    book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
    experiments = book.parse('Experiments', header=0)    
    for i, row in experiments.iterrows():
        if pandas.isnull(row['accession']) and not pandas.isnull(row['aliases:array']):
            alias = row['aliases:array'].split(',')[0]
            try:
                obj = server.get_json(alias)
                experiments.loc[i, 'accession'] = obj['accession']
                experiments.loc[i, 'uuid'] = obj['uuid']
            except requests.HTTPError as e:
                if e.response.status_code == 404:
                    print("{} not found".format(alias))

    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)
    experiments

In [17]:
print(server.server)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = book.parse('Experiments', header=0)
#experiments = experiments[experiments['accession'] != 'skip barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [18]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [19]:
# In case we need to look up previously submitted libraries by alias.
if 0:
    rep_uuids = {}
    book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
    experiments = book.parse('Experiments', header=0)
    for i, row in experiments.iterrows():
        if not pandas.isnull(row['accession']) and not row['accession'].startswith('skip'):
            if pandas.isnull(row['aliases:array']):
                aliases = []
            else:
                aliases = row['aliases:array'].split(',')

            aliases.append(row['accession'])
            try:
                obj = server.get_json(row['accession'])
                for alias in aliases:
                    for replicate in obj['replicates']:
                        key = (alias, replicate['biological_replicate_number'], replicate['technical_replicate_number'])
                        assert key not in rep_uuids, "{} already found".format(key)
                        rep_uuids[key] = replicate['uuid']
                    
            except requests.HTTPError as e:
                if e.response.status_code == 404:
                    print("{} not found".format(alias))

    replicates = book.parse('Replicates', header=0)
    for i, row in replicates.iterrows():
        key = (row['experiment'], row['biological_replicate_number:integer'], row['technical_replicate_number:integer'])
        uuid = rep_uuids.get(key)
        if pandas.isnull(row['uuid']):
            replicates.loc[i, 'uuid'] = uuid
        else:
            if row['uuid'] != uuid:
                print('row {} uuid mismatch {} != {}'.format(i, row['uuid'], uuid))
                replicates.loc[i, 'uuid'] = uuid
    
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)
    replicates

In [25]:
print(server.server)
print(spreadsheet_name)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = book.parse('Replicates', header=0)
replicates = replicates[replicates['uuid'] != 'skip barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
ENCODE4 May2019 PacBio
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-08-13T23:14:21.780474+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['ali-mortazavi:rep1_m_p18-20mo_heart'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR410XGE/', 'library': '/libraries/ENCLB607FYB/', 'status': 'in progress', '@id': '/replicates/93b913dd-3734-4ab4-8f3b-0386cc83c402/', '@type': ['Replicate', 'Item'], 'uuid': '93b913dd-3734-4ab4-8f3b-0386cc83c402'}]}
row 266 created: 93b913dd-3734-4ab4-8f3b-0386cc83c402
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-08-13T23:14:22.059511+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['ali-mortazavi:rep2_m_p18-20mo_heart'], 'schema_version': '9', 'biological_replicate_number': 2, 'technical_replicate_number': 1, 'experiment

In [26]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

In [22]:
with pandas.ExcelWriter(f'/dev/shm/submission.xlsx') as book:
    biosample.to_excel(book, sheet_name='Biosample', index=False)
    experiments.to_excel(book, sheet_name='Experiment', index=False)
    libraries.to_excel(book, sheet_name='Library', index=False)
    replicates.to_excel(book, sheet_name='Replicate', index=False)
    #file_sheet.to_excel(book, sheet_name='File', index=False)
 